# Ethereum and ERC20 derivation and transaction manual

In [1]:
from web3 import Web3, middleware
from web3.gas_strategies.time_based import fast_gas_price_strategy

from crypto_eth import HDPrivateKey, HDKey

from binascii import hexlify

import pandas as pd

from IPython.display import display
from ipywidgets import widgets, Layout

In [2]:
phrase = ""

text = widgets.Text(
    description='Phrase:',
    layout=Layout(width="70%")
)
display(text)

def handle(sender):
    global phrase
    phrase = text.value
    text.value =""
    
text.on_submit(handle)

Text(value='', description='Phrase:', layout=Layout(width='70%'))

## Derivation

In [5]:
master_key = HDPrivateKey.master_key_from_mnemonic(phrase)

del phrase

root_keys = HDKey.from_path(master_key, "m/44'/60'/0'")
acct_priv_key = root_keys[-1]
private_keys = []
addresses = []

# derivation example for Ethereum generate first 10 addresses
for i in range(10):
    keys = HDKey.from_path(acct_priv_key, '{change}/{index}'.format(change=0, index=i))
    private_key = keys[-1]
    public_key = private_key.public_key
    private_keys.append(private_key._key.to_hex())
    addresses.append(private_key.public_key.address())
    

df = pd.DataFrame(addresses, columns = ['Address']) 

df

,Address
0,0x88a007ec4f1819f24c0988fc9c26496b99b436d1
1,0xab33d517b6a63a0b1c099b8438d6641cf1a984cc
2,0xb76d3a5d568f339411a6f0697694cddb6396df58
3,0xcb64cd644d24b57a96287f61dffeb402b01e23c6
4,0x8396738b4cb2a2a7ca636161346a7cb22ee06e25
5,0xbd666c0ec86628475b1602aba0ebf45570a33d4d
6,0x46ff31e07d54a28c468c300325afcdac4c6b7f2e
7,0x44874a7d255df37f5b9c91d173bee6ab4441c881
8,0x9a2c78cad9e6c0dc46820c37704065bb22e5eab0
9,0x577c31aad54366e7f8cf931a804f409d20b6928c


## Ethereum transction
### Setup Web3

In [3]:
# use infura endpoint
# endpoint_url = 'wss://ropsten.infura.io/ws/adc06f70568e46d88376a8a3a30e5497'
endpoint_url = 'https://ropsten.infura.io/v3/adc06f70568e46d88376a8a3a30e5497'

# Ropsten chainId is 3
chainId = 3

# setup connection
w3 = Web3(Web3.HTTPProvider(endpoint_url))
if not w3.isConnected():
    print('Can\'t connect to Web3 endpoint')

# select gas price strategy
w3.eth.setGasPriceStrategy(fast_gas_price_strategy)

### Get last block

In [4]:
block = w3.eth.getBlock('latest')
print(f'Current block: {block}')

Current block: AttributeDict({'difficulty': 2473520557, 'extraData': HexBytes('0xd983010906846765746889676f312e31312e3133856c696e7578'), 'gasLimit': 8000000, 'gasUsed': 7986699, 'hash': HexBytes('0x2a482798be9e3c19ced99fb32d9aba21a974ea86f51036177b56861ea523cd9d'), 'logsBloom': HexBytes('0x041080a40000030200000000004246000008010210810000c00000201204010100240a400000000000040040004800000000000002002008002000020014000010400000048030e010000008400300540010500000040a38000622000020008800001000420000004080000014010a008000402011020a08008180100a0000100300000000000080000000000000201020049420410000848000000000400060000800030000000001800100018000000800400000802008000000108001604200947002c00008000400020000900028082000000000002800254100500260000288244080090082410000000100000080808040008108200880801210400008'), 'miner': '0x4CCfB3039b78d3938588157564c9AD559bAfAB94', 'mixHash': HexBytes('0x7b620f67b24cec870da0dab5043529df681bea277105f408984814e20b713278'), 'nonce': HexBytes('0x409188f403e6b79b'), 'numbe

### Prepare accounts

In [5]:
# take first two accounts as from and to
from_address = w3.toChecksumAddress(private_keys[0].public_key.address())
from_private_key = private_keys[0]._key.to_hex()

to_address = w3.toChecksumAddress(private_keys[1].public_key.address())
to_private_key = private_keys[1]._key.to_hex()

### Send Ethereum

In [7]:
# fetching nonce (count of transactions)
nonce = w3.eth.getTransactionCount(from_address)

transaction = {
    'to': to_address,
    'value': 200_000, # 0.0000000000002 Ether
    'gas': 21_000,
    'gasPrice': w3.eth.generateGasPrice(),
    'nonce': nonce,
    'chainId': chainId
}

signed = w3.eth.account.sign_transaction(transaction, from_private_key)

print(f'Signed transaction: {hexlify(signed.rawTransaction)}')

transaction_hash = w3.eth.sendRawTransaction(signed.rawTransaction)

print(f'Transaction hash: {hexlify(transaction_hash)}')

print(f'Ropsten link: https://ropsten.etherscan.io/tx/0x' + str(hexlify(transaction_hash), "utf-8"))

Signed transaction: b'f8688203fe845d043b0982520894ab33d517b6a63a0b1c099b8438d6641cf1a984cc83030d408029a0675b45378755960de061c23a554b47d708a50f6a7ae0ddd7bb0f18b10e2db269a00a30823cd74b9a0bbd7f7da555b4f1faa310d2fd057b0252235c1e3254f7219b'
Transaction hash: b'fe8e10d6db6f8f10acebabcf68a9466b10485d006b16c03352637885636cc620'
Ropsten link: https://ropsten.etherscan.io/tx/0xfe8e10d6db6f8f10acebabcf68a9466b10485d006b16c03352637885636cc620


### Ethereum balance

In [8]:
addresses = [w3.toChecksumAddress(key.public_key.address()) for key in private_keys]
balances = [w3.fromWei(w3.eth.getBalance(address), 'ether') for address in addresses]

df = pd.DataFrame(zip(addresses,balances), columns = ['Address', 'Balance']) 

df

,Address,Balance
0,0x88a007eC4F1819F24C0988fc9C26496b99b436D1,2.508039966300027123
1,0xab33D517b6A63A0B1C099b8438D6641cF1a984cC,0.984099864528103776
2,0xb76d3a5D568f339411a6f0697694cDDb6396df58,2.154467975998
3,0xcb64cD644D24b57a96287F61DfFeb402B01E23c6,0.998700340002
4,0x8396738b4Cb2A2a7Ca636161346a7cB22Ee06E25,3.979769
5,0xBd666C0Ec86628475b1602aBA0eBF45570A33d4d,3.01
6,0x46FF31E07D54a28c468c300325AfcdAc4c6b7F2e,4
7,0x44874A7d255dF37f5b9C91d173bEE6ab4441c881,4.1
8,0x9a2C78cad9e6c0Dc46820C37704065bb22E5Eab0,3
9,0x577C31aAd54366E7F8Cf931A804F409d20B6928C,2.99948678


### Send ERC20

In [9]:
# read abi for ERC20 contract
with open('erc20_abi.json', 'r') as file:
    erc20_abi = file.read()

In [11]:
# Another Test Coin (ATC) ERC20
erc20_contract_address = "0x96f55B7d683EB46FF23aBEf2aFb41EB54FCF524b"
erc20_contract = w3.eth.contract(address=erc20_contract_address, abi=erc20_abi)

# fetching nonce (count of transactions)
nonce = w3.eth.getTransactionCount(from_address)

erc20_transaction = erc20_contract.functions.transfer(erc20_contract_address, 1).buildTransaction(
    {
        'chainId': chainId,
        'gas': 100_000,
        'gasPrice': w3.eth.generateGasPrice(),
        'nonce': nonce,
    })

print(erc20_transaction)

signed_erc20_transaction = w3.eth.account.sign_transaction(erc20_transaction, from_private_key)

print(f'Signed transaction: {hexlify(signed_erc20_transaction.rawTransaction)}')

erc20_transaction_hash = w3.eth.sendRawTransaction(signed_erc20_transaction.rawTransaction)

print(f'Transaction hash: {hexlify(erc20_transaction_hash)}')

print(f'Ropsten link: https://ropsten.etherscan.io/tx/0x' + str(hexlify(erc20_transaction_hash), "utf-8"))

{'value': 0, 'chainId': 3, 'gas': 100000, 'gasPrice': 1541335609, 'nonce': 1023, 'to': '0x96f55B7d683EB46FF23aBEf2aFb41EB54FCF524b', 'data': '0xa9059cbb00000000000000000000000096f55b7d683eb46ff23abef2afb41eb54fcf524b0000000000000000000000000000000000000000000000000000000000000001'}
Signed transaction: b'f8ab8203ff845bdeea39830186a09496f55b7d683eb46ff23abef2afb41eb54fcf524b80b844a9059cbb00000000000000000000000096f55b7d683eb46ff23abef2afb41eb54fcf524b000000000000000000000000000000000000000000000000000000000000000129a0c6a329165da3709ce81cc6e892e8d1f230100df3f85acb64d8841da2db3c80c3a020470eb199f433a8d39fb551860f5d793c97381f7a4f09847a48c1aefc117a24'
Transaction hash: b'bf36bc09ada9a956e0cc3090209050325990b46fd58242fe08acc66417d819fe'
Ropsten link: https://ropsten.etherscan.io/tx/0xbf36bc09ada9a956e0cc3090209050325990b46fd58242fe08acc66417d819fe


### ERC20 balances

In [12]:
decimals = 10 ** erc20_contract.functions.decimals().call()

erc20_addresses = [w3.toChecksumAddress(key.public_key.address()) for key in private_keys]
erc20_balances = [(erc20_contract.functions.balanceOf(address).call() / decimals) for address in erc20_addresses]

df = pd.DataFrame(zip(erc20_addresses, erc20_balances), columns=['Address', 'Balance'])

df

,Address,Balance
0,0x88a007eC4F1819F24C0988fc9C26496b99b436D1,6681.173300
1,0xab33D517b6A63A0B1C099b8438D6641cF1a984cC,179.315696
2,0xb76d3a5D568f339411a6f0697694cDDb6396df58,217.092688
3,0xcb64cD644D24b57a96287F61DfFeb402B01E23c6,112.454000
4,0x8396738b4Cb2A2a7Ca636161346a7cB22Ee06E25,109.750000
5,0xBd666C0Ec86628475b1602aBA0eBF45570A33d4d,52.740000
6,0x46FF31E07D54a28c468c300325AfcdAc4c6b7F2e,131.410000
7,0x44874A7d255dF37f5b9C91d173bEE6ab4441c881,105.320000
8,0x9a2C78cad9e6c0Dc46820C37704065bb22E5Eab0,108.510000
9,0x577C31aAd54366E7F8Cf931A804F409d20B6928C,33.590000
